# Load Dataset

In [1]:
import requests
import io
import zipfile
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
import json
import re
import string
import html

In [2]:
! pwd

/home/xinye


In [3]:
# url = "https://cs.stanford.edu/~myasu/projects/scisumm_net/scisummnet_release1.1__20190413.zip"
# response = requests.get(url)
# with zipfile.ZipFile(io.BytesIO(response.content)) as zipObj:
#      # Extract all the contents of zip file in different directory
#      zipObj.extractall("nlp_data")
#      print("File is unzipped in nlp_data folder") 

File is unzipped in nlp_data folder


In [71]:
# get all raw text, break all papers into two parts -- Abstract and rest of document
# first get all filepaths
xmlfiles = []
citations = []
for subdir, dirs, files in os.walk(r'/home/xinye/nlp_data/scisummnet_release1.1__20190413/top1000_complete'):
    for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".xml"):
            xmlfiles.append(filepath)
        if filepath.endswith(".json"):
            citations.append(filepath)
        if filepath.endswith(".txt"):
            summary.append(filepath)
            

In [72]:
#next parse all XML documents

def parse_xml_abstract(fp):
    """ parse an XML journal article into an abstract and the rest of the text
    """
    try:
        tree = ET.parse(fp)
    except Exception as e:
        return "","",str(e)
    root = tree.getroot()
    
    ab = []
    bod = []
    
    for child in root:
        if child.tag == "ABSTRACT":
            for block in child:
                ab.append(block.text)
        else:
            for block in child:
                bod.append(block.text)
                
    #convert from list --> string
    abstract = "\n".join(ab)
    body = "\n".join(bod)
    
    #decode html entities
    abstract = html.unescape(abstract)
    body = html.unescape(body)
    
    return abstract,body,""

In [73]:
raw_cols = []
for fpn in range(len(xmlfiles)):
    ab,bod,err = parse_xml_abstract(xmlfiles[fpn])
    if err:
        #print(fp, err)
        continue
    f = open(citations[fpn]) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 
    only_text = []
    for entry in data:
        only_text.append(entry['clean_text'])
#     print(only_text)
        
    raw_cols.append([ab,bod,only_text,xmlfiles[fpn]])

df = pd.DataFrame(raw_cols, columns=["abstract","body","citations", "filepath"])

In [74]:
# get all summary text using the summary path

summary_text = []
for fpn in range(len(summary)):
    f = open(summary[fpn]) 
    data = f.read()
    summary_text.append(data)

df['summary'] = summary_text

In [77]:
# drop rows with missing values or empty abstracts
df = df.dropna()
df = df[df.abstract != ""]

In [78]:
df['paper'] = df['body'] + ' ' + df['abstract']
df.head(5)

,abstract,body,citations,filepath,summary,paper
0,2: Test set performance of our systems: and ou...,"The open source Moses (Koehn et al., 2007) MT ...",[Factored translation models have also been us...,/home/xinye/nlp_data/scisummnet_release1.1__20...,Experiments in Domain Adaptation for Statistic...,"The open source Moses (Koehn et al., 2007) MT ..."
1,"At present, adapting an Information Extraction...","Most of the world’s information is recorded, p...",[Our work is related to previous work on domai...,/home/xinye/nlp_data/scisummnet_release1.1__20...,"On-Demand Information Extraction\nAt present, ...","Most of the world’s information is recorded, p..."
2,"In this paper, we have proposed novel methods ...","In this paper, we have proposed novel methods ...",[These are much finer grained than Penn Treeba...,/home/xinye/nlp_data/scisummnet_release1.1__20...,Supertagging: An Approach To Almost Parsing\nI...,"In this paper, we have proposed novel methods ..."
3,Statistical machine translation systems are us...,"In statistical machine translation (SMT), tran...","[Transductive learning method (Ueffing et al, ...",/home/xinye/nlp_data/scisummnet_release1.1__20...,Transductive learning for statistical machine ...,"In statistical machine translation (SMT), tran..."
4,We combine the strengths of Bayesian modeling ...,Most state-of-the-art statistical machine tran...,[This is in line with earlier work on consiste...,/home/xinye/nlp_data/scisummnet_release1.1__20...,Bayesian Learning of Non-Compositional Phrases...,Most state-of-the-art statistical machine tran...


In [79]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split

# Load NLTK's stop words list
stop_words = nltk.corpus.stopwords.words('english')


# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# baseline: first sentence

In [80]:
def first_sentence_summary(text):
    sentences = nltk.sent_tokenize(text)
    if sentences:
        return sentences[0]
    else:
        return ""

In [81]:
test_df['first_sentence'] = test_df['body'].apply(lambda x: first_sentence_summary(x))

In [82]:
from rouge import Rouge
rouge = Rouge()

test_df['rouge_scores'] = test_df.apply(lambda row: rouge.get_scores(row['first_sentence'], row['summary'])[0], axis=1)

avg_rouge_scores = {
    'rouge-1': sum([s['rouge-1']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-2': sum([s['rouge-2']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-su4': sum([s['rouge-su4']['f'] for s in test_df['rouge_scores'] if 'rouge-su4' in s]) / len(test_df),
}

print(f"Average ROUGE-1 score: {avg_rouge_scores['rouge-1']:.2f}")
print(f"Average ROUGE-2 score: {avg_rouge_scores['rouge-2']:.2f}")
print(f"Average ROUGE-SU4 score: {avg_rouge_scores['rouge-su4']:.2f}")


Average ROUGE-1 score: 0.15
Average ROUGE-2 score: 0.05
Average ROUGE-SU4 score: 0.00


In [83]:

rouge = Rouge()

test_df['rouge_scores'] = test_df.apply(lambda row: rouge.get_scores(row['first_sentence'], row['summary'])[0], axis=1)

avg_rouge_scores = {
    'rouge-1': sum([s['rouge-1']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-2': sum([s['rouge-2']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-l': sum([s['rouge-l']['f'] for s in test_df['rouge_scores']]) / len(test_df),
}

print(f"Average ROUGE-1 score: {avg_rouge_scores['rouge-1']:.2f}")
print(f"Average ROUGE-2 score: {avg_rouge_scores['rouge-2']:.2f}")
print(f"Average ROUGE-L score: {avg_rouge_scores['rouge-l']:.2f}")

Average ROUGE-1 score: 0.15
Average ROUGE-2 score: 0.05
Average ROUGE-L score: 0.14


# baseline2: tfidf

In [84]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

# stop words 
stop_words = nltk.corpus.stopwords.words('english')
vectorizer = TfidfVectorizer(stop_words=stop_words)
train_matrix = vectorizer.fit_transform(train_df['body'])

def extract_top_sentences(text, num_sentences=3):
    sentences = nltk.sent_tokenize(text)
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    sentence_scores = vectorizer.fit_transform(sentences).toarray().sum(axis=1)
    
    # top-ranked sentence
    top_indices = sentence_scores.argsort()[-num_sentences:]
    top_indices.sort()
    
    summary = [sentences[i] for i in top_indices]
    return ' '.join(summary)

In [85]:
test_df['generated_summary_tfidf'] = test_df['body'].apply(lambda x: extract_top_sentences(x))

# Apply extractive summarization using TF-IDF to the "body" column
#df['body_extractive_summary_tfidf'] = df['body'].apply(lambda x: summarize_extractive_tfidf(x))

In [87]:
import pandas as pd
import nltk
from rouge import Rouge
from sklearn.model_selection import train_test_split


rouge = Rouge()
test_df['rouge_scores'] = test_df.apply(lambda row: rouge.get_scores(row['generated_summary_tfidf'], row['summary'])[0], axis=1)

avg_rouge_scores = {
    'rouge-1': sum([s['rouge-1']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-2': sum([s['rouge-2']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-l': sum([s['rouge-l']['f'] for s in test_df['rouge_scores']]) / len(test_df),
}

print(f"Average ROUGE-1 score: {avg_rouge_scores['rouge-1']:.2f}")
print(f"Average ROUGE-2 score: {avg_rouge_scores['rouge-2']:.2f}")
print(f"Average ROUGE-L score: {avg_rouge_scores['rouge-l']:.2f}")


Average ROUGE-1 score: 0.22
Average ROUGE-2 score: 0.04
Average ROUGE-L score: 0.19


In [88]:

avg_rouge_scores = {
    'rouge-1': sum([s['rouge-1']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-2': sum([s['rouge-2']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-su4': sum([s['rouge-su4']['f'] for s in test_df['rouge_scores'] if 'rouge-su4' in s]) / len(test_df),
}

print(f"Average ROUGE-1 score: {avg_rouge_scores['rouge-1']:.2f}")
print(f"Average ROUGE-2 score: {avg_rouge_scores['rouge-2']:.2f}")
print(f"Average ROUGE-SU4 score: {avg_rouge_scores['rouge-su4']:.2f}")

Average ROUGE-1 score: 0.22
Average ROUGE-2 score: 0.04
Average ROUGE-SU4 score: 0.00


# centroid baseline

In [89]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim.downloader as api

model = api.load('word2vec-google-news-300')
stop_words = stopwords.words('english')

# centroid of sentence vectors
def compute_centroid(sentence_vectors):
    return np.mean(sentence_vectors, axis=0)

# cosine similarity between two vectors
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def summarize_centroid(text, num_sentences=3):
    sentences = nltk.sent_tokenize(text)
    sentence_vectors = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence.lower())
        words = [word for word in words if word not in stop_words]
        if not words:
            continue
        
        sentence_vector = np.zeros((300,))
        for word in words:
            if word in model.key_to_index:
                sentence_vector += model.get_vector(word)
        sentence_vector /= len(words)
        sentence_vectors.append(sentence_vector)
    
    centroid_vector = compute_centroid(sentence_vectors)
    sentence_similarities = []
    for sentence_vector in sentence_vectors:
        similarity = cosine_similarity(sentence_vector, centroid_vector)
        sentence_similarities.append(similarity)

    top_indices = np.argsort(sentence_similarities)[-num_sentences:]
    top_indices.sort()
    
    summary = [sentences[i] for i in top_indices]
    return ' '.join(summary)


In [90]:
test_df['generated_summary_centroid'] = test_df['body'].apply(lambda x: summarize_centroid(x))


In [92]:
rouge = Rouge()
test_df['rouge_scores'] = test_df.apply(lambda row: rouge.get_scores(row['generated_summary_centroid'], row['summary'])[0], axis=1)


# Compute the average ROUGE scores across all papers in the test set
avg_rouge_scores = {
    'rouge-1': sum([s['rouge-1']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-2': sum([s['rouge-2']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-l': sum([s['rouge-l']['f'] for s in test_df['rouge_scores']]) / len(test_df),
}

# Print the average ROUGE scores
print(f"Average ROUGE-1 score: {avg_rouge_scores['rouge-1']:.2f}")
print(f"Average ROUGE-2 score: {avg_rouge_scores['rouge-2']:.2f}")
print(f"Average ROUGE-L score: {avg_rouge_scores['rouge-l']:.2f}")



Average ROUGE-1 score: 0.17
Average ROUGE-2 score: 0.03
Average ROUGE-L score: 0.14


# pre-trained T5

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
tokenizer = AutoTokenizer.from_pretrained('t5-small')
max_length = 50

def summarize_abstractive(text):
    segments = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    
    # Generate summary
    summaries = []
    for segment in segments:
        input_ids = tokenizer.encode("summarize: " + segment, return_tensors='pt', truncation=True, max_length=max_length)
        summary_ids = model.generate(input_ids, max_length=max_length, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    summary = ' '.join(summaries)
    return summary


In [ ]:
test_df['generated_summary_new'] = test_df['body'].apply(lambda x: summarize_abstractive(x))

In [ ]:

from rouge import Rouge
rouge = Rouge()
test_df['rouge_scores'] = test_df.apply(lambda row: rouge.get_scores(row['generated_summary_new'], row['abstract'])[0], axis=1)

# Compute the average ROUGE scores across all papers in the test set
avg_rouge_scores = {
    'rouge-1': sum([s['rouge-1']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-2': sum([s['rouge-2']['f'] for s in test_df['rouge_scores']]) / len(test_df),
    'rouge-l': sum([s['rouge-l']['f'] for s in test_df['rouge_scores']]) / len(test_df),
}

print(f"Average ROUGE-1 score: {avg_rouge_scores['rouge-1']:.2f}")
print(f"Average ROUGE-2 score: {avg_rouge_scores['rouge-2']:.2f}")
print(f"Average ROUGE-L score: {avg_rouge_scores['rouge-l']:.2f}")
